In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, multilabel_confusion_matrix, confusion_matrix
from numpy import unravel_index
from sklearn import tree

In [2]:
df =pd.read_csv("data/IRIS.csv")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
X_all = df.iloc[:, 0:4]
y_all = df.iloc[:, 4:]
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.3)

In [6]:
# cl = RandomForestClassifier(n_estimators=1, max_depth=4)
cl = tree.DecisionTreeClassifier(max_depth=3)
cl.fit(X_train, y_train)
y_pred = cl.predict(X_test)
confusion_matrix(y_pred.tolist(), y_test.iloc[:, 0].tolist())

array([[11,  0,  0],
       [ 0, 16,  1],
       [ 0,  1, 16]])

In [219]:

print(tree.export_text(cl))

|--- feature_3 <= 0.80
|   |--- class: Iris-setosa
|--- feature_3 >  0.80
|   |--- feature_3 <= 1.75
|   |   |--- feature_2 <= 4.95
|   |   |   |--- class: Iris-versicolor
|   |   |--- feature_2 >  4.95
|   |   |   |--- class: Iris-virginica
|   |--- feature_3 >  1.75
|   |   |--- class: Iris-virginica



In [38]:
import imp
import src.python.random_forest
imp.reload(src.python.random_forest)
from src.python.random_forest import RandomForestFromScratch
rf = RandomForestFromScratch(max_depth=5, n_estimators=10)
rf.fit(X_train.to_numpy(), y_train.to_numpy())

Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here
Data didn't split into two. Making terminal here


In [39]:
y_pred2 = rf.predict(X_test.values)
confusion_matrix(y_pred2.T, y_test.values)

array([[11,  0,  0],
       [ 0, 16,  2],
       [ 0,  1, 15]])

y_pred2

In [177]:
rf.root.depth_first_print()

Terminal class is Iris-setosa
Terminal class is Iris-versicolor
Terminal class is Iris-setosa
Terminal class is Iris-versicolor
Terminal class is Iris-virginica
Terminal class is Iris-virginica
Terminal class is Iris-virginica
Node depth 0 self.dim 3 self.val 2.0Node depth 1 self.dim 3 self.val 1.0Node depth 2 self.dim 0 self.val 4.4Node depth 3 self.dim None self.val NoneNode depth 3 self.dim 1 self.val 3.0Node depth 4 self.dim None self.val NoneNode depth 4 self.dim None self.val NoneNode depth 2 self.dim 2 self.val 6.3Node depth 3 self.dim 2 self.val 5.5Node depth 4 self.dim None self.val NoneNode depth 4 self.dim None self.val NoneNode depth 3 self.dim None self.val NoneNode depth 1 self.dim None self.val None


'Node depth 0 self.dim 3 self.val 2.0Node depth 1 self.dim 3 self.val 1.0Node depth 2 self.dim 0 self.val 4.4Node depth 3 self.dim None self.val NoneNode depth 3 self.dim 1 self.val 3.0Node depth 4 self.dim None self.val NoneNode depth 4 self.dim None self.val NoneNode depth 2 self.dim 2 self.val 6.3Node depth 3 self.dim 2 self.val 5.5Node depth 4 self.dim None self.val NoneNode depth 4 self.dim None self.val NoneNode depth 3 self.dim None self.val NoneNode depth 1 self.dim None self.val None'

In [154]:
pd.DataFrame({"y_test": y_test.values[:, 0], "ref_y_pred": y_pred, "out_y_pred": y_pred2})

,y_test,ref_y_pred,out_y_pred
0,Iris-versicolor,Iris-versicolor,Iris-setosa
1,Iris-virginica,Iris-virginica,Iris-virginica
2,Iris-setosa,Iris-setosa,Iris-setosa
3,Iris-virginica,Iris-virginica,Iris-virginica
4,Iris-versicolor,Iris-versicolor,Iris-virginica
5,Iris-versicolor,Iris-versicolor,Iris-setosa
6,Iris-setosa,Iris-setosa,Iris-setosa
7,Iris-virginica,Iris-versicolor,Iris-virginica
8,Iris-versicolor,Iris-versicolor,Iris-virginica
9,Iris-virginica,Iris-virginica,Iris-virginica


array(['Iris-versicolor', 'Iris-virginica', 'Iris-setosa',
       'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-setosa', 'Iris-virginica', 'Iris-versicolor',
       'Iris-virginica', 'Iris-virginica', 'Iris-virginica',
       'Iris-versicolor', 'Iris-setosa', 'Iris-virginica'], dtype=object)